In [1]:
import datasets
import numpy as np
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification

c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset("ktgiahieu/maccrobat2018_2020")
dataset = dataset['train'].train_test_split(test_size=0.3)
test_validation = dataset.pop('test')
test_validation = test_validation.train_test_split(test_size=0.5)
valid = test_validation.pop('train')
test = test_validation.pop('test')
dataset['validation'] = valid
dataset['test'] = test
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 280
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 60
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 60
    })
})

In [3]:
label_list = []

with open('./label_list.txt', 'r') as f:
    for line in f:
        parts = line.split(':')
        label = parts[1].strip()
        
        label_list.append(label)

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [5]:
idx_to_label = {i: label for i, label in enumerate(label_list)}
label_to_idx = {label: i for i, label in enumerate(label_list)}

In [6]:
def tags_to_indices(example):
    example['tags'] = [label_to_idx[tag] for tag in example['tags']]
    return example

In [7]:
def indices_to_tags(example):
    example['tags'] = [idx_to_label[idx] for idx in example['tags']]
    return example

In [8]:
dataset = dataset.map(lambda x: tags_to_indices(x))

Map: 100%|██████████| 60/60 [00:00<00:00, 1732.61 examples/s]


In [9]:
def tokenize_and_align_labels(example, label_all_tokens=True):
    tokenized_input = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(example['tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_input["labels"] = labels
    return tokenized_input

In [10]:
q = tokenize_and_align_labels(dataset['train'][0:1])
len(q['input_ids'][0]), len(q['labels'][0])

(388, 388)

In [11]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 60/60 [00:00<00:00, 692.82 examples/s]


In [12]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    'test-ner',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
)

In [14]:
metric = datasets.load_metric('seqeval')

C:\Users\kevin\AppData\Local\Temp\ipykernel_2736\520486469.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('seqeval')
c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(p):
    logits, label_ids = p
    predictions = np.argmax(logits, axis=2)

    # Flatten the batches and remove the special tokens
    true_predictions = [
        label_list[p] for batch_preds, batch_labels in zip(predictions, label_ids) 
        for p, l in zip(batch_preds, batch_labels) if l != -100
    ]

    true_labels = [
        label_list[l] for batch_labels in label_ids
        for l in batch_labels if l != -100
    ]

    # Calculate metrics
    precision = precision_score([true_labels], [true_predictions])
    recall = recall_score([true_labels], [true_predictions])
    f1 = f1_score([true_labels], [true_predictions])
    accuracy = accuracy_score([true_labels], [true_predictions])

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }


In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
trainer.train()

model.save_pretrained('ner_model')

tokenizer.save_pretrained('tokenizer')

  1%|          | 9/900 [00:07<10:44,  1.38it/s]c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                               
  1%|          | 9/900 [00:08<10:44,  1.38it/s]

{'eval_loss': 2.286665678024292, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6304816179025644, 'eval_runtime': 0.625, 'eval_samples_per_second': 95.993, 'eval_steps_per_second': 3.2, 'epoch': 1.0}


                                                
  2%|▏         | 18/900 [00:15<10:28,  1.40it/s]

{'eval_loss': 2.0101654529571533, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6304816179025644, 'eval_runtime': 0.6238, 'eval_samples_per_second': 96.19, 'eval_steps_per_second': 3.206, 'epoch': 2.0}


                                                
  3%|▎         | 27/900 [00:22<10:25,  1.40it/s]

...

                                                 
 99%|█████████▉| 891/900 [12:17<00:06,  1.42it/s]

{'eval_loss': 0.3120429217815399, 'eval_precision': 0.8588684043229498, 'eval_recall': 0.8840176672664812, 'eval_f1': 0.8712615880693269, 'eval_accuracy': 0.9425602891097367, 'eval_runtime': 0.6261, 'eval_samples_per_second': 95.826, 'eval_steps_per_second': 3.194, 'epoch': 99.0}


                                                 
100%|██████████| 900/900 [12:25<00:00,  1.21it/s]


{'eval_loss': 0.3116986155509949, 'eval_precision': 0.8593476531424026, 'eval_recall': 0.8835269098642238, 'eval_f1': 0.8712695596063882, 'eval_accuracy': 0.942490791576899, 'eval_runtime': 0.6181, 'eval_samples_per_second': 97.064, 'eval_steps_per_second': 3.235, 'epoch': 100.0}
{'train_runtime': 745.2519, 'train_samples_per_second': 37.571, 'train_steps_per_second': 1.208, 'train_loss': 0.4934656312730577, 'epoch': 100.0}


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [19]:
import json

config = json.load(open('ner_model/config.json'))

In [20]:
config['id2label'] = idx_to_label
config['label2id'] = label_to_idx

json.dump(config, open('ner_model/config.json', 'w'))

model_fine_tuned = AutoModelForTokenClassification.from_pretrained('ner_model')

In [24]:
def evaluate_ner_model_on_test(trainer, test_dataset):
    evaluation_results = trainer.evaluate(test_dataset)

    return {
        'test_loss': evaluation_results.get('eval_loss'),
        'test_precision': evaluation_results.get('eval_precision'),
        'test_recall': evaluation_results.get('eval_recall'),
        'test_f1': evaluation_results.get('eval_f1'),
        'test_accuracy': evaluation_results.get('eval_accuracy')
    }

In [29]:
test_results = evaluate_ner_model_on_test(trainer, tokenized_dataset['test'])
print(test_results)

100%|██████████| 2/2 [00:00<00:00,  6.20it/s]

{'test_loss': 0.649170458316803, 'test_precision': 0.7491681191570274, 'test_recall': 0.7823928512328314, 'test_f1': 0.7654201068479844, 'test_accuracy': 0.8866003708073488}
